In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch

In [2]:
S = open('failed_rec_segs.txt').readlines()
S = [i.split(')')[0][1:].split(', ') for i in S]
S = [(int(i), int(j)) for i, j in S]

In [22]:
prepare_data._serialize((3058025, 146))

RuntimeError: Error : flac decoder lost sync.

In [11]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
filtered = metadata.reindex(metadata.index & prepare_data.get_downloaded())
#     filtered = filter_downloaded(metadata)
filtered = filtered[filtered['sig_len'] > prepare_data.CHUNK_SKIP_SIZE]
print(len(filtered), 'records')

379772 records


In [12]:
len(S)

1018

In [14]:
os.path.isdir

<function genericpath.isfile(path)>

In [24]:
os.makedirs('tmp')
os.rename('test.txt', 'tmp/test2.txt')

In [18]:
os.rename?

Signature: os.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)
Docstring:
Rename a file or directory.

If either src_dir_fd or dst_dir_fd is not None, it should be a file
  descriptor open to a directory, and the respective path string (src or dst)
  should be relative; the path will then be relative to that directory.
src_dir_fd and dst_dir_fd, may not be implemented on your platform.
  If they are unavailable, using them will raise a NotImplementedError.
Type:      builtin_function_or_method


In [5]:
import glob

In [6]:
files = glob.glob('/scr-ssd/mimic/waveforms/*')

In [68]:
files[:5]

['/scr-ssd/mimic/waveforms/3327804_0097_0027.tfrec',
 '/scr-ssd/mimic/waveforms/3283596_0009_0000.tfrec',
 '/scr-ssd/mimic/waveforms/3213904_0012_0104.tfrec',
 '/scr-ssd/mimic/waveforms/3277543_0112_0031.tfrec',
 '/scr-ssd/mimic/waveforms/3140902_0004_0266.tfrec']

In [50]:
files[0][:-11] + '/' + files[0][-10:]

'/scr-ssd/mimic/waveforms/3769026_0023/0074.tfrec'

In [57]:
shutil.move?

Signature: shutil.move(src, dst, copy_function=<function copy2 at 0x7f0462344290>)
Docstring:
Recursively move a file or directory to another location. This is
similar to the Unix "mv" command. Return the file or directory's
destination.

If the destination is a directory or a symlink to a directory, the source
is moved inside the directory. The destination path must not already
exist.

If the destination already exists but is not a directory, it may be
overwritten depending on os.rename() semantics.

If the destination is on our current filesystem, then rename() is used.
Otherwise, src is copied to the destination and then removed. Symlinks are
recreated under the new name if os.rename() fails because of cross
filesystem renames.

The optional `copy_function` argument is a callable that will be used
to copy the source or it will be delegated to `copytree`.
By default, copy2() is used, but any function that supports the same
signature (like copy()) can be used.

A lot more could be don

In [7]:
files = sorted(files)
files = [i for i in files if 'tfrec' in i]

In [75]:
files[:10]

['/scr-ssd/mimic/waveforms/3000003_0005_0000.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0001.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0002.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0003.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0004.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0005.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0006.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0007.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0008.tfrec',
 '/scr-ssd/mimic/waveforms/3000003_0005_0009.tfrec']

In [9]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')

In [19]:
index_names = ['rec_id', 'seg_id', 'sig_name']
columns = index_names + ['sig_index', 'baseline', 'adc_gain']
sig_data = pandas.read_hdf('/scr-ssd/mimic/sig_data_backup.hdf', columns=columns)

In [20]:
sig_data

,rec_id,sig_name,sig_index,baseline,adc_gain
0,3000003,II,0,0,24.0
1,3000003,V,1,0,24.0
2,3000003,II,0,0,45.0
3,3000003,V,1,0,24.0
4,3000003,II,0,0,29.0
...,...,...,...,...,...
8847983,3999988,RESP,0,307,409.0
8847984,3999988,PLETH,1,0,1023.0
8847985,3999988,III,2,255,512.0
8847986,3999988,V,3,255,512.0


In [18]:
ls /scr-ssd/mimic

metadata.csv          metadata_old.hdf     sig_data.hdf          waveforms/
metadata.hdf          partition.pkl        sig_data_matched.csv
metadata_matched.csv  sig_data_backup.hdf  sig_data_old.hdf
metadata_old_2.hdf    sig_data.csv         test_subject_ids.txt


In [12]:
sig_data.drop_duplicates(index_names, inplace=True)
sig_data = sig_data.astype({'sig_name': str})
dtypes = sig_data.dtypes
sig_data.set_index(index_names, inplace=True)
sig_data.at[:, 'sig_index'] += 1
sig_data.at[:, 'subject_id'] = subject_ids
sig_data.fillna({'subject_id': -1}, inplace=True)
sig_data = sig_data.astype({'subject_id': 'int32'})
sig_data.reset_index(inplace=True)
sig_data.set_index(['subject_id'] + index_names, inplace=True, verify_integrity=True)
sig_data = sig_data.unstack(fill_value=0)
sig_data = sig_data.astype({(k, s): dtypes[k] for k, s in sig_data})

KeyError: Index(['seg_id'], dtype='object')

In [10]:
metadata

hadm_id  subject_id                    time gender        dob  \
rec_id  seg_id                                                                  
3000003 1        173995        9950 2155-04-19 19:44:07.664      M 2108-02-01   
        2        173995        9950 2155-04-19 19:46:22.685      M 2108-02-01   
        3        173995        9950 2155-04-19 19:46:25.757      M 2108-02-01   
        4        173995        9950 2155-04-19 19:46:33.981      M 2108-02-01   
        5        173995        9950 2155-04-19 19:46:37.053      M 2108-02-01   
...                 ...         ...                     ...    ...        ...   
3999988 6            -1          -1                     NaT    NaN        NaT   
        7            -1          -1                     NaT    NaN        NaT   
        8            -1          -1                     NaT    NaN        NaT   
        9            -1          -1                     NaT    NaN        NaT   
        10           -1          -1                     NaT    NaN        NaT   

                age  weight  height  sig_len death_time   race  
rec_id  seg_id                                                  
3000003 1        47     NaN      74    16878        NaT  white  
        2        47     NaN      74      384        NaT  white  
        3        47     NaN      74     1028        NaT  white  
        4        47     NaN      74      384        NaT  white  
        5        47     NaN      74   438826        NaT  white  
...             ...     ...     ...      ...        ...    ...  
3999988 6        -1     NaN      -1     5250        NaT    NaN  
        7        -1     NaN      -1      250        NaT    NaN  
        8        -1     NaN      -1      500        NaT    NaN  
        9        -1     NaN      -1      250        NaT    NaN  
        10       -1     NaN      -1    24875        NaT    NaN  

[2273298 rows x 11 columns]

In [8]:
for i in files:
    if os.path.isfile(i):
        if not os.path.isdir(i[:-11]):
            os.makedirs(i[:-11])
        os.rename(i, i[:-11] + '/' + i[-10:])

OSError: [Errno 28] No space left on device: '/scr-ssd/mimic/waveforms/3000189_0010'

In [69]:
tmp = [i for i in files if 'tfrec' not in i]
len(tmp)

82

In [81]:
os.listdir(tmp[1])

['0041.tfrec']

In [53]:
os.remove(files[-1])

In [88]:
for i in files[-2000:-1000]:
    os.remove(i)

In [21]:
os.makedirs('tmp')

In [25]:
! cat tmp/test2.txt

(3653853, 23) failed
(3160159, 10) failed
(3793645, 5) failed
(3043848, 29) failed
(3975239, 26) failed
(3851968, 82) failed
(3527787, 123) failed
(3619557, 4) failed
(3643142, 23) failed
(3515232, 8) failed
(3090785, 18) failed
(3879773, 8) failed
(3103011, 17) failed
(3174440, 67) failed
(3364035, 17) failed
(3551187, 13) failed
(3979128, 2) failed
(3464937, 8) failed
(3688201, 28) failed
(3524201, 3) failed
(3144888, 10) failed
(3320495, 15) failed
(3367600, 24) failed
(3477377, 1) failed
(3519644, 1) failed
(3542049, 104) failed
(3023488, 122) failed
(3282838, 14) failed
(3984927, 4) failed
(3230691, 8) failed
(3419353, 13) failed
(3561851, 9) failed
(3875809, 8) failed
(3504983, 49) failed
(3900416, 19) failed
(3475838, 8) failed
(3338100, 25) failed
(3644175, 26) failed
(3589453, 46) failed
(3115752, 15) failed
(3464995, 17) failed
(3294308, 137) failed
(3213502, 9) failed
(3517865, 20) failed
(3137077, 6) failed
(3055173, 108) failed
(3033149, 24) failed
(3651295, 10) failed
(32

In [23]:
! ls

align_metadata.ipynb		 old
check_initial_predictions.ipynb  plot_batch.ipynb
csv_to_hdf.ipynb		 prepare_pressures.ipynb
data_formats.ipynb		 rsync_file_list.ipynb
failed_rec_segs.txt		 scratch.ipynb
fix_distribution.ipynb		 split_flacs.ipynb
generate_report.ipynb		 test_metrics.ipynb
include_non_matched.ipynb	 test.txt
index_clinical_data.ipynb	 Untitled1.ipynb
integrate_new_data.ipynb	 Untitled.ipynb
new_data_counts.ipynb
